In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import datetime

In [4]:
def create_fundsinfo(website_address):
    # url to soup object
    soup = BeautifulSoup(requests.get(website_address).content, "html.parser")
    
    # get the column from the html page that we want, and extract the href links for the funds
    navigation_col = soup.find(class_ = "nav-col col-1")
    
    dict_ = {}
    
    for link in navigation_col.find_all('a'):
        today = datetime.date.today()
        dict_['Date'] = today
        
        if '#' not in link.get('href'):
            fund_title, market_price = get_fund_price(link.get('href'))
            if fund_title in dict_.keys():
                dict_[fund_title].append(market_price)
            else:
                dict_[fund_title] = [market_price]
                
    df = pd.DataFrame(dict_)
    export_csv = df.to_csv(r"C:\\Users\\Brett\\Desktop\\fundsinfo.csv", index = None, header = True)

In [39]:
create_fundsinfo('https://smartshares.co.nz/types-of-funds')

In [5]:
def get_fund_price(website_address):
    # url to soup object
    soup = BeautifulSoup(requests.get(website_address).content, 'html.parser')
    
    # get the name of the fund
    fund_title = soup.title.getText()
    fund_title = re.search('(.+?)ETF', fund_title)[0]
    
    # get the fund details
    fund = soup.find(class_ = 'container fund-details')
    fund_details = fund.find(class_ = 'details')
    
    # get the market price of the fund
    market_price = list(fund_details.children)[3].find(class_ = "data").getText()
    
    return fund_title, float(market_price.replace('$',''))

def update_fundsinfo(file_location, website_address):  
    ''' use this function after the create_fundsinfo() function to update the fundsinfo.csv file'''
    # url to soup object
    soup = BeautifulSoup(requests.get(website_address).content, "html.parser")
    
    # get the column from the html page that we want, and extract the href links for the funds
    navigation_col = soup.find(class_ = "nav-col col-1")
    
    dict_ = {}
    
    for link in navigation_col.find_all('a'):
        today = datetime.date.today()
        dict_['Date'] = today
        
        if '#' not in link.get('href'):
            fund_title, market_price = get_fund_price(link.get('href'))
            if fund_title in dict_.keys():
                dict_[fund_title].append(market_price)
            else:
                dict_[fund_title] = [market_price]
                
    df = pd.DataFrame(dict_)
    df.to_csv(file_location, header=False, mode = 'a', index = None)

In [8]:
update_fundsinfo("C:\\Users\\Brett\\Desktop\\fundsinfo.csv", "https://smartshares.co.nz/types-of-funds")